#

#

#

#

#

#

# 함수 데커레이터와 클로저

## 데커레이터 기본 지식

### 일반적으로 데커레이터는 함수를 다른 함수로 대체한다

In [5]:
def deco(func):
    def inner():
        print('running inner()')
    return inner # deco()가 inner()함수 객체를 반환한다.

In [6]:
@deco
def target():  # target()을 deco로 데커레이트했다.
    print('running target()')

In [7]:
target() # 데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다.

running inner()


In [8]:
target #조사해보면 target이 inner()를 가리키고 있음을 알 수 있다.

<function __main__.deco.<locals>.inner()>

## 파이썬이 데커레이터를 실행하는 시점

### 예제 7-2 registration.py 모듈

In [10]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->',registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()

running register(<function f1 at 0x0000020DDD4DB9C0>)
running register(<function f2 at 0x0000020DDD4D82C0>)
running main()
registry -> [<function f1 at 0x0000020DDD4DB9C0>, <function f2 at 0x0000020DDD4D82C0>]
running f1()
running f2()
running f3()


## 데커레이터로 개선한 전략 패턴

### 예제 7-3 promotion 데커레이터로 채운 promos 리스트

In [17]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() *.1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인 받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

In [18]:
# 255p